In [1]:
# packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as scio
from tqdm import tqdm

In [2]:
# transfer matlab struct data to pandas dataframe
data ,videotimes = scio.loadmat('research data/vpnList.mat',struct_as_record=True)['vpnList'], pd.read_csv('research data/Videotimes.csv')
keys = ['curVP','currentTime','gazeX','gazeY','pupilPos_combined','pupilDiameter_combined','gyroX','gyroY','gyroZ','accX','accY','accZ','confused']
videotimes

,Participant,Landmark,start in sec,end in sec,confused,during in sec
0,1,1,0.00000,1.56060,0,1.56060
1,1,2,4.14413,5.18391,0,1.03978
2,1,3,12.05848,17.30456,0,5.24608
3,1,4,19.71280,21.26494,0,1.55214
4,1,5,23.96891,26.59987,0,2.63096
...,...,...,...,...,...,...
175,15,8,59.98350,69.91180,0,9.92830
176,15,9,71.86372,86.67927,0,14.81555
177,15,10,88.94365,93.38355,0,4.43990
178,15,11,95.81585,103.83981,0,8.02396


In [3]:
n, samp_list = 0, []
for i in tqdm(range(data.shape[0])):
    samp_list.append(data[i][0][1].shape[0])
    n += data[i][0][1].shape[0]
n,samp_list

100%|██████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<?, ?it/s]


(542072,
 [6048,
  35555,
  208,
  53027,
  62589,
  66136,
  16221,
  26053,
  11255,
  78040,
  52700,
  38452,
  36500,
  49078,
  10210])

# Data labeled and cleaning
datalist = []
for i in tqdm(range(data.shape[0])):
    vpnlist, prt_list, idx_ldlist = data[i][0][1].copy(), [], []
    vpnlist[:,1] /= 1000000
    for idx_vp in range(vpnlist.shape[0]):
        idx_landmark = np.where((videotimes['Participant']==i+1).values*(vpnlist[idx_vp,1]>videotimes['start in sec']).values*(vpnlist[idx_vp,1]<=videotimes['end in sec']).values)[0]
        if len(idx_landmark)!=0:
            prt_list.append(idx_vp)
            idx_ldlist += idx_landmark.tolist()
    vpnlist[prt_list,-1] = videotimes.loc[idx_ldlist,'confused'].values
    datalist.append(pd.DataFrame(vpnlist[prt_list,:],columns=keys))
np.sum([datalist[i].shape[0] for i in range(len(datalist))]),[datalist[i].iloc[:,-1].sum() for i in range(len(datalist))]

In [4]:
# Data labeled and cleaning, and window sequence = 1 second
datalist = []
for i in tqdm(range(data.shape[0])):
    vpnlist, prt_list, labels = data[i][0][1].copy(), [], []
    vpnlist[:,1] /= 1000000
    for idx_vp in range(vpnlist.shape[0]):
        idx_landmark = np.where((videotimes['Participant']==i+1).values*(vpnlist[idx_vp,1]>=videotimes['start in sec']-1).values*(vpnlist[idx_vp,1]<=videotimes['end in sec']+1).values)[0]
        if len(idx_landmark)!=0:
            prt_list.append(idx_vp)
            labels.append(max(videotimes.loc[idx_landmark,'confused'].values))
    vpnlist[prt_list,-1] = labels
    datalist.append(pd.DataFrame(vpnlist[prt_list,:],columns=keys))

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [04:39<00:00, 18.61s/it]


In [5]:
np.sum([datalist[i].shape[0] for i in range(len(datalist))]),[datalist[i].iloc[:,-1].sum() for i in range(len(datalist))]

(381324,
 [0.0,
  0.0,
  4.0,
  35488.0,
  14786.0,
  14212.0,
  0.0,
  0.0,
  0.0,
  46433.0,
  0.0,
  0.0,
  12034.0,
  0.0,
  0.0])

In [7]:
vp_sequence = pd.concat(datalist)
#vp_sequence.to_csv('vp_sequence.csv')
data_confusion = pd.read_csv('vp_sequence.csv',index_col=0)
data_confusion

,curVP,currentTime,gazeX,gazeY,pupilPos_combined,pupilDiameter_combined,gyroX,gyroY,gyroZ,accX,accY,accZ,confused
0,1.0,0.005165,855.0,363.0,66.936884,4.030,-4.20,0.3125,2.2586,-1.5298,-9.7258,-1.3166,0.0
1,1.0,0.015154,856.0,363.0,66.646658,4.040,-4.27,0.4584,1.4313,-1.5298,-9.7712,-1.2848,0.0
2,1.0,0.025146,856.0,363.0,66.946883,4.030,-4.48,0.3692,1.1313,-1.5298,-9.7712,-1.2848,0.0
3,1.0,0.035138,856.0,364.0,66.656880,4.035,-5.11,0.3813,1.0624,-1.5298,-9.8030,-1.3302,0.0
4,1.0,0.045135,856.0,364.0,66.946883,4.030,-5.60,0.3124,1.0502,-1.5298,-9.7962,-1.3689,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6438,15.0,113.540250,1038.0,233.0,70.692379,3.070,-0.77,-1.2895,0.0569,0.1961,-9.8688,-0.5054,0.0
6439,15.0,113.550039,1037.0,230.0,70.933039,3.055,-0.42,-0.9205,-0.0201,0.1569,-9.8302,-0.4985,0.0
6440,15.0,113.559828,1036.0,228.0,70.682380,3.075,-0.14,-0.2028,-0.4622,0.0785,-9.7848,-0.5304,0.0
6441,15.0,113.605432,1035.0,229.0,70.922850,3.055,0.00,1.0703,-1.2329,0.1961,-9.9461,-0.5190,0.0
